# Autor
##**Aucapuri Corimanya Wilger Fabricio 200821**

# Requerimientos / Librerias

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 72.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install num2words


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=449ea2266c3634bc683a73a9c5cd493a77af5bb5186bbe6e3f7fee4b4457f6b8
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import requests
import numpy as np
import pandas as pd
import re
import spacy
import nltk
from itertools import islice
from num2words import num2words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.internals import find_jars_within_path
from nltk.tag import StanfordPOSTagger
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Descargar recursos necesarios de nltk
nltk.download('punkt')
nltk.download('stopwords')

# Cargar el modelo de Spacy
nlp = spacy.load("es_core_news_sm")

# Definir stopwords en español
stop_words = set(stopwords.words('spanish'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Lectura del URL

In [ ]:
# Almacenenamos la URL relevante en una variable en Python para luego hacer una solicitud GET
url = 'https://www.gutenberg.org/cache/epub/21282/pg21282-images.html'

In [ ]:
# Hacer la solicitud de tipo GET
r = requests.get(url)
# Verificamos el tipo de objeto
type(r)


requests.models.Response

# Verificando contenido

In [ ]:
# Extraemos HTML del objeto
html = r.text
# Mostraremos 2000 caracteres de la cadena
print('%.100000s' % html)

<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"><style>
#pg-header div, #pg-footer div {
    all: initial;
    display: block;
    margin-top: 1em;
    margin-bottom: 1em;
    margin-left: 2em;
}
#pg-footer div.agate {
    font-size: 90%;
    margin-top: 0;
    margin-bottom: 0;
    text-align: center;
}
#pg-footer li {
    all: initial;
    display: block;
    margin-top: 1em;
    margin-bottom: 1em;
    text-indent: -0.6em;
}
#pg-footer div.secthead {
    font-size: 110%;
    font-weight: bold;
}
#pg-footer #project-gutenberg-license {
    font-size: 110%;
    margin-top: 0;
    margin-bottom: 0;
    text-align: center;
}
#pg-header-heading {
    all: inherit;
    text-align: center;
    font-size: 120%;
    font-weight:bold;
}
#pg-footer-heading {
    all: inherit;
    text-align: center;
    font-size: 120%;
    font-weight: normal;
    margin-top: 0;
    margin-bottom: 0;
}
#pg-header #pg-machine-header p {
    text-indent: -4em;
    margin-left: 4em;
    margin-top:

# Eliminando comentarios

In [ ]:
# Expresión regular para capturar comentarios HTML
comment_pattern = r'<!--.*?-->'

# Reemplazar comentarios con una cadena vacía
html = re.sub(comment_pattern, '', html, flags=re.DOTALL)

print('%.100000s' % html)

<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8"><style>
#pg-header div, #pg-footer div {
    all: initial;
    display: block;
    margin-top: 1em;
    margin-bottom: 1em;
    margin-left: 2em;
}
#pg-footer div.agate {
    font-size: 90%;
    margin-top: 0;
    margin-bottom: 0;
    text-align: center;
}
#pg-footer li {
    all: initial;
    display: block;
    margin-top: 1em;
    margin-bottom: 1em;
    text-indent: -0.6em;
}
#pg-footer div.secthead {
    font-size: 110%;
    font-weight: bold;
}
#pg-footer #project-gutenberg-license {
    font-size: 110%;
    margin-top: 0;
    margin-bottom: 0;
    text-align: center;
}
#pg-header-heading {
    all: inherit;
    text-align: center;
    font-size: 120%;
    font-weight:bold;
}
#pg-footer-heading {
    all: inherit;
    text-align: center;
    font-size: 120%;
    font-weight: normal;
    margin-top: 0;
    margin-bottom: 0;
}
#pg-header #pg-machine-header p {
    text-indent: -4em;
    margin-left: 4em;
    margin-top:

# Extraccion de los titulos de los cuentos y el indicador del final

In [ ]:
# Expresión regular para capturar todo el contenido entre <h2> y </h2> o <div class="footnotes"><h3>NOTAS:</h3>
matches = re.findall(r'<h2>.*?</h2>|<div class="footnotes"><h3>NOTAS:</h3>', html, re.DOTALL)

# Imprimir cada coincidencia encontrada
for i in matches:
    print(i)

<h2> <a id="LOS_DUENDES_DEL_CUZCO"></a><a href="#toc" class="pginternal">LOS DUENDES DEL CUZCO</a></h2>
<h2> <a id="LOS_POLVOS_DE_LA_CONDESA"></a><a href="#toc" class="pginternal">LOS POLVOS DE LA CONDESA</a></h2>
<h2> <a id="EL_JUSTICIA_MAYOR_DE_LAYCACOTA"></a><a href="#toc" class="pginternal">EL JUSTICIA MAYOR DE LAYCACOTA</a></h2>
<h2> <a id="RACIMO_DE_HORCA"></a><a href="#toc" class="pginternal">RACIMO DE HORCA</a></h2>
<h2> <a id="AMOR_DE_MADRE"></a><a href="#toc" class="pginternal">AMOR DE MADRE</a></h2>
<h2> <a id="LUCAS_EL_SACRILEGO"></a><a href="#toc" class="pginternal">LUCAS EL SACRÍLEGO</a></h2>
<h2> <a id="RUDAMENTE_PULIDAMENTE_MANOSAMENTE"></a><a href="#toc" class="pginternal">RUDAMENTE, PULIDAMENTE, MAÑOSAMENTE</a></h2>
<h2> <a id="EL_RESUCITADO"></a><a href="#toc" class="pginternal">EL RESUCITADO</a></h2>
<h2> <a id="EL_CORREGIDOR_DE_TINTA"></a><a href="#toc" class="pginternal">EL CORREGIDOR DE TINTA</a></h2>
<h2> <a id="LA_GATITA_DE_MARI-RAMOS_QUE_HALAGA_CON_LA_COLA_Y_A

# Recuperar el contedido entre titulos

In [ ]:
# Ahora recuperamos el contenido entre cada par de títulos
contents = []

# Para cada par de títulos, obtenemos el contenido entre ellos
for i in range(len(matches) - 1):  # No incluimos el último título
    # Creamos una expresión regular para capturar el contenido entre los títulos consecutivos
    pattern = re.escape(matches[i]) + r'(.*?)' + re.escape(matches[i+1])
    content_between = re.search(pattern, html, re.DOTALL)

    if content_between:
        contents.append(content_between.group(1).strip())

# Imprimir los títulos y sus contenidos correspondientes
for idx, (title, content) in enumerate(zip(matches[:-1], contents), 1):
    print(f"Título {idx}: {title}")
    print(f"Contenido {idx}: {content}\n")
    print("\n" + "-"*40 + "\n")

Título 1: <h2> <a id="LOS_DUENDES_DEL_CUZCO"></a><a href="#toc" class="pginternal">LOS DUENDES DEL CUZCO</a></h2>
Contenido 1: <p class="c"><span class="smcap">crónica que trata de cómo el virrey poeta entendía la justicia</span></p>
<p>Esta tradición no tiene otra fuente de autoridad que el relato del
pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún
cronista hace mención de ella, y sólo en un manuscrito de rápidas
apuntaciones, que abarca desde la época del virrey marqués de Salinas
hasta la del duque de la Palata, encuentro las siguientes líneas:</p>
<p>«En este tiempo del gobierno del príncipe de Squillace, murió malamente
en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por
el descomulgado».</p>
<p>Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en
los <i>Anales del Cuzco</i>, que posee inéditos el señor obispo de Ochoa,
tampoco se avanza más, sino que el misterioso suceso está colocado en
época diversa a la que yo le asi

## Verificar la correctra extraccion de un capitulo

In [ ]:
print(contents[0])

<p class="c"><span class="smcap">crónica que trata de cómo el virrey poeta entendía la justicia</span></p>
<p>Esta tradición no tiene otra fuente de autoridad que el relato del
pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún
cronista hace mención de ella, y sólo en un manuscrito de rápidas
apuntaciones, que abarca desde la época del virrey marqués de Salinas
hasta la del duque de la Palata, encuentro las siguientes líneas:</p>
<p>«En este tiempo del gobierno del príncipe de Squillace, murió malamente
en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por
el descomulgado».</p>
<p>Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en
los <i>Anales del Cuzco</i>, que posee inéditos el señor obispo de Ochoa,
tampoco se avanza más, sino que el misterioso suceso está colocado en
época diversa a la que yo le asigno.</p>
<p>Y he tenido en cuenta para preferir los tiempos de don Francisco de
Borja; y Aragón, no sólo la apuntación ya citad

# Limpieza de los titulos

In [ ]:
# Expresión regular para extraer los títulos
pattern = r'<a[^>]*class="pginternal"[^>]*>(.*?)</a>'

# Extraer los títulos
titles = []
for html in matches[:len(matches)-1]:
    match = re.search(pattern, html)
    if match:
        titles.append(match.group(1))
    else:
        titles.append('No match found')

for i in titles:
    print(i)



LOS DUENDES DEL CUZCO
LOS POLVOS DE LA CONDESA
EL JUSTICIA MAYOR DE LAYCACOTA
RACIMO DE HORCA
AMOR DE MADRE
LUCAS EL SACRÍLEGO
RUDAMENTE, PULIDAMENTE, MAÑOSAMENTE
EL RESUCITADO
EL CORREGIDOR DE TINTA
LA GATITA DE MARI-RAMOS QUE HALAGA CON LA COLA Y ARAÑA CON LAS MANOS
¡A LA CÁRCEL TODO CRISTO!
NADIE SE MUERE HASTA QUE DIOS QUIERE
EL FRAILE Y LA MONJA DEL CALLAO
POR BEBER UNA COPA DE ORO
UNA EXCOMUNION FAMOSA
ACEITUNA, UNA
OFICIOSIDAD NO AGRADECIDA
EL ALMA DE FRAY VENANCIO
LA TRENZA DE SUS CABELLOS
DE ASTA Y REJON
LOS ARGUMENTOS DEL CORREGIDOR
LA NIÑA DEL ANTOJO
LA LLORONA DEL VIERNES SANTO
¡A NADAR, PECES!
CONVERSION DE UN LIBERTINO
EL REY DEL MONTE
TRES CUESTIONES HISTORICAS SOBRE PIZARRO


## Liempieza de los capitulos

In [ ]:
# Función para limpiar contenido HTML no deseado
def clean_html(content):
    # Primero reemplazar los saltos de línea por espacios
    content = content.replace('\n', ' ')

    # Expresión regular para eliminar las etiquetas no deseadas
    pattern = r'</?span[^>]*>|</?i>|<br\s*/?>|<a\s+id="[^"]*"></a>|<a\s+class="fnanchor\s+pginternal"[^>]*>[^<]*</a>|<sup>|</sup>|<p style="margin-left: 40%;">|class="r"|class="c"|\r'

    # Aplicar la expresión regular para eliminar las etiquetas HTML
    return re.sub(pattern, '', content)

# Limpiar y convertir el contenido
cleaned_contents = [clean_html(content).strip() for content in contents]

# Mostrar el contenido limpio
for content in cleaned_contents:
    print(content)
    print("\n" + "-"*40 + "\n")


<p >crónica que trata de cómo el virrey poeta entendía la justicia</p> <p>Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas:</p> <p>«En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado».</p> <p>Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno.</p> <p>Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo la apuntación ya citada, sino la especialísima circunstancia de q

Se dejan indicadores de parrafos y titulos para facilitar la divicion de estos mas adelante

# Crear corpus como Diccionario

##Juntar contenidos en una sola variable

Juntamos la los titulos con su respectivo contenido / capitulos

In [ ]:
# titles
# cleaned_contents
# Crear una lista donde se combinan títulos y contenidos
result = [f"{title}\n{content}" for title, content in zip(titles, cleaned_contents)]

# Imprimir la lista resultante
for item in result:
    print(item)


LOS DUENDES DEL CUZCO
<p >crónica que trata de cómo el virrey poeta entendía la justicia</p> <p>Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas:</p> <p>«En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado».</p> <p>Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno.</p> <p>Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo la apuntación ya citada, sino la especialís

## Crear corpus de diccionario de diccionarios

In [ ]:
def split_by_roman_numerals(text):
    # Expresión regular para separar por números romanos dentro de <h3> y </h3>
    sections = re.split(r'(<h3>\s*[IVXLCDM]+\s*</h3>)', text)
    result = {}

    # Guardar el texto introductorio en la clave 0
    if sections[0].strip():
        result[0] = sections[0].strip()

    # Guardar los capítulos con números romanos (sin las etiquetas <h3>) como claves
    for i in range(1, len(sections)-1, 2):
        roman_numeral = re.sub(r'<.*?>', '', sections[i]).strip()  # Eliminar las etiquetas <h3> y </h3>
        chapter_text = sections[i+1].strip()
        result[roman_numeral] = chapter_text

    return result

# Crear el diccionario de diccionarios
title_content_dict = {}
for title, content in zip(titles, cleaned_contents):
    title_content_dict[title] = split_by_roman_numerals(content)


In [ ]:
# Imprimir el diccionario resultante
for title, chapters in title_content_dict.items():
    print(f"{title}: {chapters}")

print(len(title_content_dict))

LOS DUENDES DEL CUZCO: {0: '<p >crónica que trata de cómo el virrey poeta entendía la justicia</p> <p>Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas:</p> <p>«En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado».</p> <p>Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno.</p> <p>Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo la apuntación ya citada, sino la espe

# 6.Modulos solicitados

In [ ]:
def eliminar_etiquetas_p(texto):
    # Expresión regular para eliminar las etiquetas <p> y </p>
    texto_limpio = re.sub(r'</?p[^>]*>', '', texto)

    return texto_limpio

##a. Listar los relatos contenidos en el corpus (similar al índice de un libro)


In [ ]:
# Módulo a: Listar los relatos contenidos en el corpus (similar al índice de un libro)
def listar_relatos(corpus_dict):
    return list(corpus_dict.keys())

In [ ]:
relatos = listar_relatos(title_content_dict)
for relato in relatos:
    print(relato)
print (len(relatos))

LOS DUENDES DEL CUZCO
LOS POLVOS DE LA CONDESA
EL JUSTICIA MAYOR DE LAYCACOTA
RACIMO DE HORCA
AMOR DE MADRE
LUCAS EL SACRÍLEGO
RUDAMENTE, PULIDAMENTE, MAÑOSAMENTE
EL RESUCITADO
EL CORREGIDOR DE TINTA
LA GATITA DE MARI-RAMOS QUE HALAGA CON LA COLA Y ARAÑA CON LAS MANOS
¡A LA CÁRCEL TODO CRISTO!
NADIE SE MUERE HASTA QUE DIOS QUIERE
EL FRAILE Y LA MONJA DEL CALLAO
POR BEBER UNA COPA DE ORO
UNA EXCOMUNION FAMOSA
ACEITUNA, UNA
OFICIOSIDAD NO AGRADECIDA
EL ALMA DE FRAY VENANCIO
LA TRENZA DE SUS CABELLOS
DE ASTA Y REJON
LOS ARGUMENTOS DEL CORREGIDOR
LA NIÑA DEL ANTOJO
LA LLORONA DEL VIERNES SANTO
¡A NADAR, PECES!
CONVERSION DE UN LIBERTINO
EL REY DEL MONTE
TRES CUESTIONES HISTORICAS SOBRE PIZARRO
27


##b. Recuperar el texto de un relato buscándolo por su título.


In [ ]:
def recuperar_relato_por_titulo(corpus_dict, titulo):
    titulo= titulo.upper()
    if titulo in corpus_dict:
        # Obtener todos los capítulos del relato (diccionario de capítulos)
        capitulos = corpus_dict[titulo]

        # Unir los capítulos en un solo texto, manteniendo el número del capítulo
        relato_completo = []
        for capitulo, contenido in capitulos.items():
            # Agregar el número o indicador del capítulo seguido por su contenido
            if capitulo == 0:
                # Para el texto introductorio (clave 0), no agregar el número de capítulo
                relato_completo.append(contenido)
            else:
                relato_completo.append(f"{capitulo}\n{contenido}")

        # Unir los capítulos limpios en un solo texto con doble salto de línea entre ellos
        return "\n\n".join(relato_completo)

    else:
        return f"Relato con el título '{titulo}' no encontrado."


In [ ]:
t= "lOs DUENDES del CUZCO"
relato_completo = eliminar_etiquetas_p(recuperar_relato_por_titulo(title_content_dict, t))

print(relato_completo)


crónica que trata de cómo el virrey poeta entendía la justicia Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas: «En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado». Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno. Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo la apuntación ya citada, sino la especialísima circunstancia de que, conocido el carácter del vir

##c. Recuperar el texto de un relato, especificando su título y capítulo.


In [ ]:
def int_to_roman(num):
    val = [
        1000, 900, 500, 400,
        100, 90, 50, 40,
        10, 9, 5, 4,
        1
        ]
    syb = [
        "M", "CM", "D", "CD",
        "C", "XC", "L", "XL",
        "X", "IX", "V", "IV",
        "I"
        ]
    roman_num = ''
    i = 0
    while num > 0:
        for _ in range(num // val[i]):
            roman_num += syb[i]
            num -= val[i]
        i += 1
    return roman_num


In [ ]:
def recuperar_capitulo_por_titulo_y_capitulo(corpus_dict, titulo, numero_capitulo):
    if titulo in corpus_dict:
        capitulos = corpus_dict[titulo]

        # Convertir el número de capítulo a la clave correspondiente
        if numero_capitulo == 0:
            # Para el texto introductorio
            clave_capitulo = 0
        else:
            # Convertir el número de capítulo a número romano
            try:
                clave_capitulo = int_to_roman(numero_capitulo)
            except ValueError:
                return f"El número de capítulo {numero_capitulo} no es válido."

        # Verificar si la clave del capítulo existe
        if clave_capitulo in capitulos:
            return capitulos[clave_capitulo]
        else:
            return f"Capítulo {numero_capitulo} no encontrado en el relato '{titulo}'."
    else:
        return f"Relato con el título '{titulo}' no encontrado."


In [ ]:
t= "DE ASTA Y REJON"
n=3
capitulo = eliminar_etiquetas_p(recuperar_capitulo_por_titulo_y_capitulo(title_content_dict, t, n))

print(t,"- CAP ",n,": \n\n",capitulo)

DE ASTA Y REJON - CAP  3 : 

 Todos se hicieron lenguas del arrojo de doña Feliciana, y en Lima no se hablaba de otra cosa. De haber habido periódicos, la habrían consagrado estrepitoso bombo en la crónica local. La fama de su hazaña la había precedido a Ica, adonde llegó una mañana, armada de asta y rejón, y abocándose a su marido le dijo: —A Lima, señor mío, y a su casa si no quiere usted que haga en su personita otro tanto de lo que hice en la de Vilches, y lo deje tal que no sirva ni para simiente de rábanos. El de Mesía tembló como azogado, mandó ensillar la mula y, sin chistar ni mistar, obedeció el precepto. Desde entonces ella llevó en la casa los pantalones, y él fué el más fiel de los maridos de que hacen mención las historias sagradas y profanas, como que sabía que le iba la pelleja en el primer tropezón en que lo pillase madama. Mucho cuento es tener por compañera una mujer de asta y rejón.


d. Tokenizar un relato.


In [ ]:
def tokenizar_texto(t):
    words = word_tokenize(t)
    return words

In [ ]:
def tokenizar_texto(texto):
    # Aquí va tu implementación de tokenización
    import re
    return re.findall(r'\b\w+\b', texto)

# Recuperar el relato completo
t = "LOS DUENDES DEL CUZCO"
texto_relato = eliminar_etiquetas_p(recuperar_relato_por_titulo(title_content_dict, t))

# Tokenizar el texto del relato
words = tokenizar_texto(texto_relato)

# Mostrar las palabras tokenizadas y la cantidad de palabras
print(words)
print(len(words))


['crónica', 'que', 'trata', 'de', 'cómo', 'el', 'virrey', 'poeta', 'entendía', 'la', 'justicia', 'Esta', 'tradición', 'no', 'tiene', 'otra', 'fuente', 'de', 'autoridad', 'que', 'el', 'relato', 'del', 'pueblo', 'Todos', 'la', 'conocen', 'en', 'el', 'Cuzco', 'tal', 'como', 'hoy', 'la', 'presento', 'Ningún', 'cronista', 'hace', 'mención', 'de', 'ella', 'y', 'sólo', 'en', 'un', 'manuscrito', 'de', 'rápidas', 'apuntaciones', 'que', 'abarca', 'desde', 'la', 'época', 'del', 'virrey', 'marqués', 'de', 'Salinas', 'hasta', 'la', 'del', 'duque', 'de', 'la', 'Palata', 'encuentro', 'las', 'siguientes', 'líneas', 'En', 'este', 'tiempo', 'del', 'gobierno', 'del', 'príncipe', 'de', 'Squillace', 'murió', 'malamente', 'en', 'el', 'Cuzco', 'a', 'manos', 'del', 'diablo', 'el', 'almirante', 'de', 'Castilla', 'conocido', 'por', 'el', 'descomulgado', 'Como', 'se', 've', 'muy', 'poca', 'luz', 'proporcionan', 'estas', 'líneas', 'y', 'me', 'afirman', 'que', 'en', 'los', 'Anales', 'del', 'Cuzco', 'que', 'posee',

e. Dividir un relato en párrafos.


In [ ]:
def extraer_parrafos(texto):
    parrafos = re.findall(r'<p>(.*?)</p>', texto, re.DOTALL)
    return parrafos

In [ ]:
t = "LOS DUENDES DEL CUZCO"
texto_relato = recuperar_relato_por_titulo(title_content_dict, t)

parrafos = extraer_parrafos(texto_relato)
for p in parrafos:
    print(p)
    print("\n" + "-"*40 + "\n")


Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas:

----------------------------------------

«En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado».

----------------------------------------

Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno.

----------------------------------------

Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo la apuntación ya citada, sino l

f. Dividir la obra completa en párrafos


In [ ]:
def procesar_relatos(title_content_dict, n):
    relatos = listar_relatos(title_content_dict)

    for relato in relatos[:n]:
        print(f"Título: {relato}\n")
        texto_relato = recuperar_relato_por_titulo(title_content_dict, relato)
        parrafos = extraer_parrafos(texto_relato)

        for p in parrafos:
            print(p)
            print("\n" + "-"*40 + "\n")

# Llamada a la función
procesar_relatos(title_content_dict,5)


Título: LOS DUENDES DEL CUZCO

Esta tradición no tiene otra fuente de autoridad que el relato del pueblo. Todos la conocen en el Cuzco tal como hoy la presento. Ningún cronista hace mención de ella, y sólo en un manuscrito de rápidas apuntaciones, que abarca desde la época del virrey marqués de Salinas hasta la del duque de la Palata, encuentro las siguientes líneas:

----------------------------------------

«En este tiempo del gobierno del príncipe de Squillace, murió malamente en el Cuzco, a manos del diablo, el almirante de Castilla, conocido por el descomulgado».

----------------------------------------

Como se ve, muy poca luz proporcionan estas líneas, y me afirman que en los Anales del Cuzco, que posee inéditos el señor obispo de Ochoa, tampoco se avanza más, sino que el misterioso suceso está colocado en época diversa a la que yo le asigno.

----------------------------------------

Y he tenido en cuenta para preferir los tiempos de don Francisco de Borja; y Aragón, no sólo 

g. Tokenizar el texto de un párrafo en palabras.


In [ ]:
t = "LOS DUENDES DEL CUZCO"
texto_relato = recuperar_relato_por_titulo(title_content_dict, t)

parrafos = extraer_parrafos(texto_relato)
words = tokenizar_texto(parrafos[0])

print(words)
print(len(words))



['Esta', 'tradición', 'no', 'tiene', 'otra', 'fuente', 'de', 'autoridad', 'que', 'el', 'relato', 'del', 'pueblo', 'Todos', 'la', 'conocen', 'en', 'el', 'Cuzco', 'tal', 'como', 'hoy', 'la', 'presento', 'Ningún', 'cronista', 'hace', 'mención', 'de', 'ella', 'y', 'sólo', 'en', 'un', 'manuscrito', 'de', 'rápidas', 'apuntaciones', 'que', 'abarca', 'desde', 'la', 'época', 'del', 'virrey', 'marqués', 'de', 'Salinas', 'hasta', 'la', 'del', 'duque', 'de', 'la', 'Palata', 'encuentro', 'las', 'siguientes', 'líneas']
59


h. Tokenizar el texto de la obra completa en palabras.

In [ ]:
def procesar_relatos_tokenizados(title_content_dict,n):
    relatos = listar_relatos(title_content_dict)

    for relato in relatos[:n]:
        print(f"Título: {relato}\n")
        texto_relato = eliminar_etiquetas_p(recuperar_relato_por_titulo(title_content_dict, relato))

        # Tokenizar todo el contenido del relato
        tokens = tokenizar_texto(texto_relato)

        print(f"Tokens: {tokens}")
        print(f"Número de tokens: {len(tokens)}")
        print("\n" + "-"*40 + "\n")

# Llamada a la función
procesar_relatos_tokenizados(title_content_dict,27)


Título: LOS DUENDES DEL CUZCO

Tokens: ['crónica', 'que', 'trata', 'de', 'cómo', 'el', 'virrey', 'poeta', 'entendía', 'la', 'justicia', 'Esta', 'tradición', 'no', 'tiene', 'otra', 'fuente', 'de', 'autoridad', 'que', 'el', 'relato', 'del', 'pueblo', 'Todos', 'la', 'conocen', 'en', 'el', 'Cuzco', 'tal', 'como', 'hoy', 'la', 'presento', 'Ningún', 'cronista', 'hace', 'mención', 'de', 'ella', 'y', 'sólo', 'en', 'un', 'manuscrito', 'de', 'rápidas', 'apuntaciones', 'que', 'abarca', 'desde', 'la', 'época', 'del', 'virrey', 'marqués', 'de', 'Salinas', 'hasta', 'la', 'del', 'duque', 'de', 'la', 'Palata', 'encuentro', 'las', 'siguientes', 'líneas', 'En', 'este', 'tiempo', 'del', 'gobierno', 'del', 'príncipe', 'de', 'Squillace', 'murió', 'malamente', 'en', 'el', 'Cuzco', 'a', 'manos', 'del', 'diablo', 'el', 'almirante', 'de', 'Castilla', 'conocido', 'por', 'el', 'descomulgado', 'Como', 'se', 've', 'muy', 'poca', 'luz', 'proporcionan', 'estas', 'líneas', 'y', 'me', 'afirman', 'que', 'en', 'los', 'A

#7. Crear lo siguientes módulos independientes para normalizar un texto

In [ ]:
#ejemplo para los modulos
print(capitulo)
print(len(capitulo))

Todos se hicieron lenguas del arrojo de doña Feliciana, y en Lima no se hablaba de otra cosa. De haber habido periódicos, la habrían consagrado estrepitoso bombo en la crónica local. La fama de su hazaña la había precedido a Ica, adonde llegó una mañana, armada de asta y rejón, y abocándose a su marido le dijo: —A Lima, señor mío, y a su casa si no quiere usted que haga en su personita otro tanto de lo que hice en la de Vilches, y lo deje tal que no sirva ni para simiente de rábanos. El de Mesía tembló como azogado, mandó ensillar la mula y, sin chistar ni mistar, obedeció el precepto. Desde entonces ella llevó en la casa los pantalones, y él fué el más fiel de los maridos de que hacen mención las historias sagradas y profanas, como que sabía que le iba la pelleja en el primer tropezón en que lo pillase madama. Mucho cuento es tener por compañera una mujer de asta y rejón.
885


##a. Quitar signos de puntuación


In [ ]:
def quitar_puntuacion(texto):
    return re.sub(r'[^\w\s]', '', texto)


In [ ]:
qp=quitar_puntuacion(capitulo)
print(qp)
print(len(qp))

Todos se hicieron lenguas del arrojo de doña Feliciana y en Lima no se hablaba de otra cosa De haber habido periódicos la habrían consagrado estrepitoso bombo en la crónica local La fama de su hazaña la había precedido a Ica adonde llegó una mañana armada de asta y rejón y abocándose a su marido le dijo A Lima señor mío y a su casa si no quiere usted que haga en su personita otro tanto de lo que hice en la de Vilches y lo deje tal que no sirva ni para simiente de rábanos El de Mesía tembló como azogado mandó ensillar la mula y sin chistar ni mistar obedeció el precepto Desde entonces ella llevó en la casa los pantalones y él fué el más fiel de los maridos de que hacen mención las historias sagradas y profanas como que sabía que le iba la pelleja en el primer tropezón en que lo pillase madama Mucho cuento es tener por compañera una mujer de asta y rejón
864


##b. Convertir a minúsculas


In [ ]:
def convertir_a_minusculas(texto):
    return texto.lower()

In [ ]:
cm=convertir_a_minusculas(capitulo)
print(cm)
print(len(cm))

todos se hicieron lenguas del arrojo de doña feliciana, y en lima no se hablaba de otra cosa. de haber habido periódicos, la habrían consagrado estrepitoso bombo en la crónica local. la fama de su hazaña la había precedido a ica, adonde llegó una mañana, armada de asta y rejón, y abocándose a su marido le dijo: —a lima, señor mío, y a su casa si no quiere usted que haga en su personita otro tanto de lo que hice en la de vilches, y lo deje tal que no sirva ni para simiente de rábanos. el de mesía tembló como azogado, mandó ensillar la mula y, sin chistar ni mistar, obedeció el precepto. desde entonces ella llevó en la casa los pantalones, y él fué el más fiel de los maridos de que hacen mención las historias sagradas y profanas, como que sabía que le iba la pelleja en el primer tropezón en que lo pillase madama. mucho cuento es tener por compañera una mujer de asta y rejón.
885


##c. Quitar stop-words


In [ ]:
def quitar_stop_words(texto):
    palabras = texto.split()
    return ' '.join([palabra for palabra in palabras if palabra not in stop_words])

In [ ]:
qsw=quitar_stop_words(capitulo)
print(qsw)
print(len(qsw))

Todos hicieron lenguas arrojo doña Feliciana, Lima hablaba cosa. De haber periódicos, consagrado estrepitoso bombo crónica local. La fama hazaña precedido Ica, adonde llegó mañana, armada asta rejón, abocándose marido dijo: —A Lima, señor mío, casa si quiere usted haga personita hice Vilches, deje tal sirva simiente rábanos. El Mesía tembló azogado, mandó ensillar mula y, chistar mistar, obedeció precepto. Desde entonces llevó casa pantalones, fué fiel maridos hacen mención historias sagradas profanas, sabía iba pelleja primer tropezón pillase madama. Mucho cuento tener compañera mujer asta rejón.
604


##d. Quitar palabras con extensión mínima


In [ ]:
def quitar_palabras_cortas(texto, longitud_minima):
    palabras = texto.split()
    return ' '.join([palabra for palabra in palabras if len(palabra) >= longitud_minima])

In [ ]:
qpcem=quitar_palabras_cortas(capitulo,3)
print(qpcem)
print(len(qpcem))

Todos hicieron lenguas del arrojo doña Feliciana, Lima hablaba otra cosa. haber habido periódicos, habrían consagrado estrepitoso bombo crónica local. fama hazaña había precedido Ica, adonde llegó una mañana, armada asta rejón, abocándose marido dijo: Lima, señor mío, casa quiere usted que haga personita otro tanto que hice Vilches, deje tal que sirva para simiente rábanos. Mesía tembló como azogado, mandó ensillar mula sin chistar mistar, obedeció precepto. Desde entonces ella llevó casa los pantalones, fué más fiel los maridos que hacen mención las historias sagradas profanas, como que sabía que iba pelleja primer tropezón que pillase madama. Mucho cuento tener por compañera una mujer asta rejón.
707


##e. Lematizar


In [ ]:
nlp = spacy.load('es_core_news_sm')

def lematizar(texto):
    doc = nlp(texto)
    return ' '.join([token.lemma_ for token in doc])


In [ ]:
lema=lematizar(capitulo)
print(lema)
print(len(lema))

todo él hacer lengua del arrojo de doña Feliciana , y en Lima no él hablar de otro cosa . de haber haber periódico , él haber consagrar estrepitoso bombo en el crónica local . el fama de su hazaña él haber preceder a Ica , adonde llegar uno mañana , armado de asta y rejón , y abocar él a su marido él decir : — A Lima , señor mío , y a su casa si no querer usted que hacer en su personita otro tanto de él que hacer en el de Vilches , y él deje tal que no servir ni para simiente de rábano . el de Mesía temblar como azogado , mandar ensillar el mula y , sin chistar ni mistar , obedecer el precepto . desde entonces él llevar en el casa el pantalón , y él fué el más fiel de el marido de que hacer mención el historia sagrado y profana , como que saber que él ir el pelleja en el primero tropezón en que él pillar madama . mucho cuento ser tener por compañera uno mujer de asta y rejón .
889


##f. Eliminar duplicados

In [ ]:
def eliminar_duplicados(texto):
    palabras = texto.split()
    return ' '.join(sorted(set(palabras), key=palabras.index))

In [ ]:
ed=eliminar_duplicados(capitulo)
print(ed)
print(len(ed))

Todos se hicieron lenguas del arrojo de doña Feliciana, y en Lima no hablaba otra cosa. De haber habido periódicos, la habrían consagrado estrepitoso bombo crónica local. La fama su hazaña había precedido a Ica, adonde llegó una mañana, armada asta rejón, abocándose marido le dijo: —A Lima, señor mío, casa si quiere usted que haga personita otro tanto lo hice Vilches, deje tal sirva ni para simiente rábanos. El Mesía tembló como azogado, mandó ensillar mula y, sin chistar mistar, obedeció el precepto. Desde entonces ella llevó los pantalones, él fué más fiel maridos hacen mención las historias sagradas profanas, sabía iba pelleja primer tropezón pillase madama. Mucho cuento es tener por compañera mujer rejón.
718


In [ ]:
# Funciones de las tres secuencias de procesamiento
def secuencia_1(texto):
    texto = convertir_a_minusculas(texto)
    texto = quitar_puntuacion(texto)
    texto = quitar_stop_words(texto)
    texto = quitar_palabras_cortas(texto,3)
    texto = lematizar(texto)
    texto = eliminar_duplicados(texto)
    return texto

def secuencia_2(texto):
    texto = quitar_puntuacion(texto)
    texto = convertir_a_minusculas(texto)
    texto = lematizar(texto)
    texto = quitar_stop_words(texto)
    texto = quitar_palabras_cortas(texto,3)
    texto = eliminar_duplicados(texto)
    return texto

def secuencia_3(texto):
    texto = quitar_palabras_cortas(texto,3)
    texto = lematizar(texto)
    texto = convertir_a_minusculas(texto)
    texto = quitar_puntuacion(texto)
    texto = quitar_stop_words(texto)
    texto = eliminar_duplicados(texto)
    return texto

# Probar las secuencias
resultados = {
    'Original': capitulo,
    'Secuencia 1': secuencia_1(capitulo),
    'Secuencia 2': secuencia_2(capitulo),
    'Secuencia 3': secuencia_3(capitulo),
}

# Crear DataFrame comparativo
df = pd.DataFrame({
    'Texto Procesado': resultados.values(),
    'Tamaño del Vocabulario': [len(set(texto.split())) for texto in resultados.values()]
}, index=resultados.keys())

# Mostrar el DataFrame
print(df)


                                               Texto Procesado  \
Original     Todos se hicieron lenguas del arrojo de doña F...   
Secuencia 1  hacer lengua arrojo doña felicián limo hablar ...   
Secuencia 2  hacer lengua arrojo doña felicián lima hablar ...   
Secuencia 3  hacer lengua arrojo doña feliciana lima hablar...   

             Tamaño del Vocabulario  
Original                        116  
Secuencia 1                      71  
Secuencia 2                      69  
Secuencia 3                      69  


Aunque Secuencia 2 y Secuencia 3 parecen reducir más el vocabulario, Secuencia 1 es recomendable para una mayor precisión, ya que se realiza un filtrado progresivo con menos transformaciones tempranas que pueden eliminar palabras útiles. Sin embargo, si la reducción del tamaño del vocabulario es el objetivo principal, entonces Secuencia 2 o Secuencia 3 serían más adecuadas.

##Combinando modulos y comparando resultados

In [ ]:
def convertir_numeros_a_palabras(texto):
    # Utilizar una expresión regular para encontrar todos los números en el texto
    texto_con_numeros = re.sub(r'\d+', lambda x: num2words(int(x.group()), lang='es'), texto)
    return texto_con_numeros


In [ ]:
def normalizar_cuentos(title_content_dict):
    cuentos_normalizados = {}
    for titulo, capitulos in title_content_dict.items():
        # Crear un texto concatenado de todos los capítulos del cuento
        texto_concatenado = ' '.join(
            secuencia_1(convertir_numeros_a_palabras(eliminar_etiquetas_p(contenido)))
            for capitulo_num, contenido in capitulos.items()
        )
        cuentos_normalizados[titulo] = texto_concatenado
    return cuentos_normalizados

# Suponiendo que `title_content_dict` está definido
# Aplicar normalización
cuentos_normalizados = normalizar_cuentos(title_content_dict)

In [ ]:
# Imprimir el diccionario resultante
for title, chapters in cuentos_normalizados.items():
    print(f"{title}: {chapters}")

print(len(cuentos_normalizados))


LOS DUENDES DEL CUZCO: crónico tratar cómo virrey poeta entendía justicia tradición fuente autoridad relato pueblo conocer cuzco tal hoy presento ninguno cronista hacer mención sólo manuscrito rápido apuntación abarcar época marqués salinas duque palatar encontrar siguiente línea tiempo gobierno príncipe squillacir morir malamente cocer mano diablo almirante castilla conocido descomulgado poco luz proporcionar líneas afirmar anal poseer inédito señor obispir ochoo tampoco avanzar sino misterioso suceso colocado diverso asigno contar preferir don francisco borja aragón citado especialísimo circunstancia carácter propio espiritual palabra terminar leyenda hecho salvedad anterior descargo conciencia poner punto redondo entro materia don francisco borja aragón príncipe esquilache conde mayalde natural madrid caballero orden santiago montés contar treinta dos año felipe iii estimar nombrar virrey perú cortesano criticar nombramiento sólo ocupado entonces escribir verso galanteo desafío cuyo

In [ ]:
muestra =dict(islice(cuentos_normalizados.items(), 10))
print(muestra)

{'LOS DUENDES DEL CUZCO': 'crónico tratar cómo virrey poeta entendía justicia tradición fuente autoridad relato pueblo conocer cuzco tal hoy presento ninguno cronista hacer mención sólo manuscrito rápido apuntación abarcar época marqués salinas duque palatar encontrar siguiente línea tiempo gobierno príncipe squillacir morir malamente cocer mano diablo almirante castilla conocido descomulgado poco luz proporcionar líneas afirmar anal poseer inédito señor obispir ochoo tampoco avanzar sino misterioso suceso colocado diverso asigno contar preferir don francisco borja aragón citado especialísimo circunstancia carácter propio espiritual palabra terminar leyenda hecho salvedad anterior descargo conciencia poner punto redondo entro materia don francisco borja aragón príncipe esquilache conde mayalde natural madrid caballero orden santiago montés contar treinta dos año felipe iii estimar nombrar virrey perú cortesano criticar nombramiento sólo ocupado entonces escribir verso galanteo desafío 

## Módulo para Binary BoW

In [ ]:
def obtener_binary_bow(corpus):
    vectorizer = CountVectorizer(binary=True)
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer.get_feature_names_out()


In [ ]:
X_binary, vocab_binary = obtener_binary_bow(muestra.values())
print("Binary BoW:\n", X_binary.toarray())
print("Vocabulario Binary BoW:", vocab_binary)

Binary BoW:
 [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
Vocabulario Binary BoW: ['abadesa' 'abajo' 'abalanzar' ... 'únicamente' 'único' 'útilmente']


## Módulo para Frequency BoW

In [ ]:
def obtener_frequency_bow(corpus):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer.get_feature_names_out()


In [ ]:
X_freq, vocab_frequency = obtener_frequency_bow(muestra.values())
print("\nFrequency BoW:\n", X_freq.toarray())
print("Vocabulario Frequency BoW:", vocab_frequency)


Frequency BoW:
 [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]
 ...
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]
Vocabulario Frequency BoW: ['abadesa' 'abajo' 'abalanzar' ... 'únicamente' 'único' 'útilmente']


## Módulo para TF-IDF

In [ ]:
def obtener_tfidf(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer.get_feature_names_out()


In [ ]:
X_tfidf, vocab_tfidf = obtener_tfidf(muestra.values())
print("\nTF-IDF:\n", X_tfidf.toarray())
print("Vocabulario TF-IDF:", vocab_tfidf)


TF-IDF:
 [[0.         0.         0.         ... 0.         0.0281793  0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.03433372 0.         0.04038827]
 ...
 [0.         0.         0.         ... 0.03635805 0.031809   0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.01986012 0.        ]]
Vocabulario TF-IDF: ['abadesa' 'abajo' 'abalanzar' ... 'únicamente' 'único' 'útilmente']


##Vocabulario etiquetado (PoS)

In [ ]:
# Definimos la ubicación del archivo JAR que contiene el etiquetador POS Tagging de Stanford.
jar = "stanford-postagger-4.2.0.jar"

# Definimos la ubicación del archivo de modelo que el etiquetador utilizará.
tagger_file = "spanish-ud.tagger"

# Creamos una nueva instancia de 'StanfordPOSTagger' con la ubicación del archivo de modelo y el archivo JAR.
tagger = StanfordPOSTagger(tagger_file, jar)

# Definimos el corpus que deseas etiquetar, que corresponde a los textos en el diccionario 'muestra'.
corpus = list(muestra.values())

# Función para etiquetar el corpus usando el StanfordPOSTagger.
def etiquetar_corpus(corpus):
    etiquetas = []
    for doc in corpus:
        # Utilizamos la función 'word_tokenize' de NLTK para dividir cada documento en palabras individuales.
        words = nltk.word_tokenize(doc)
        # Etiquetamos cada documento utilizando el etiquetador POS de Stanford.
        tags = tagger.tag(words)
        etiquetas.append(tags)
    return etiquetas

# Etiquetamos el corpus y obtenemos los resultados.
tags_corpus = etiquetar_corpus(corpus)

# Imprimimos las palabras etiquetadas de cada documento.
for i, tags in enumerate(tags_corpus):
    print(f"\nDocumento {i+1} etiquetado:")
    print(tags)



Documento 1 etiquetado:
[('crónico', 'ADJ'), ('tratar', 'AUX'), ('cómo', 'PRON'), ('virrey', 'VERB'), ('poeta', 'NOUN'), ('entendía', 'VERB'), ('justicia', 'NOUN'), ('tradición', 'NOUN'), ('fuente', 'NOUN'), ('autoridad', 'NOUN'), ('relato', 'NOUN'), ('pueblo', 'NOUN'), ('conocer', 'VERB'), ('cuzco', 'NOUN'), ('tal', 'DET'), ('hoy', 'NOUN'), ('presento', 'VERB'), ('ninguno', 'PRON'), ('cronista', 'NOUN'), ('hacer', 'VERB'), ('mención', 'NOUN'), ('sólo', 'ADV'), ('manuscrito', 'ADJ'), ('rápido', 'ADJ'), ('apuntación', 'NOUN'), ('abarcar', 'VERB'), ('época', 'NOUN'), ('marqués', 'NOUN'), ('salinas', 'ADJ'), ('duque', 'NOUN'), ('palatar', 'VERB'), ('encontrar', 'VERB'), ('siguiente', 'ADJ'), ('línea', 'NOUN'), ('tiempo', 'NOUN'), ('gobierno', 'NOUN'), ('príncipe', 'NOUN'), ('squillacir', 'VERB'), ('morir', 'VERB'), ('malamente', 'ADV'), ('cocer', 'VERB'), ('mano', 'NOUN'), ('diablo', 'NOUN'), ('almirante', 'NOUN'), ('castilla', 'ADJ'), ('conocido', 'ADJ'), ('descomulgado', 'ADJ'), ('poco

##Lista de adjetivos, verbos y sustantivos.

In [ ]:
# Función para filtrar adjetivos, verbos y sustantivos
def filtrar_palabras(tags_corpus):
    adjetivos = []
    verbos = []
    sustantivos = []

    # Definir etiquetas para adjetivos, verbos y sustantivos en español
    etiquetas_adjetivos = ['ADJ']
    etiquetas_verbos = ['VERB']
    etiquetas_sustantivos = ['NOUN', 'PROPN']

    # Recorrer cada conjunto de tags en el corpus
    for tags in tags_corpus:
        for palabra, tag in tags:
            if tag in etiquetas_adjetivos:
                adjetivos.append(palabra)
            elif tag in etiquetas_verbos:
                verbos.append(palabra)
            elif tag in etiquetas_sustantivos:
                sustantivos.append(palabra)

    return adjetivos, verbos, sustantivos



In [ ]:
# Filtrar adjetivos, verbos y sustantivos
adjetivos, verbos, sustantivos = filtrar_palabras(tags_corpus)

# Mostrar resultados
print("Adjetivos:", adjetivos)
print("Verbos:", verbos)
print("Sustantivos:", sustantivos)


Adjetivos: ['crónico', 'manuscrito', 'rápido', 'salinas', 'siguiente', 'castilla', 'conocido', 'descomulgado', 'anal', 'inédito', 'ochoo', 'misterioso', 'colocado', 'diverso', 'francisco', 'citado', 'especialísimo', 'propio', 'espiritual', 'hecho', 'anterior', 'redondo', 'borja', 'esquilache', 'mayalde', 'natural', 'caballero', 'santiago', 'montés', 'perú', 'cortesano', 'ocupado', 'verso', 'galanteo', 'joven', 'india', 'cabezo', 'fuerte', 'amagado', 'filibustera', 'buen', 'faltábalar', 'brío', 'jorge', 'spitberg', 'holandés', 'chile', 'español', 'veintidós', 'reñido', 'feroz', 'cerro', 'azul', 'cañete', 'malo', 'consiguiente', 'saqueo', 'limo', 'verdadero', 'invadida', 'débil', 'bravo', 'divino', 'herej', 'anciano', 'corajudo', 'veinticincocuatrociento', 'contestado', 'paita', 'peralto', 'fundada', 'poemo', 'santo', 'luctuoso', 'cristiano', 'retirada', 'veinticuatro', 'suntuoso', 'cabildo', 'esplendidez', 'primero', 'audacia', 'filibustero', 'jacobo', 'lheremite', 'formidable', 'piráti

## Lista de nombres de entidades

In [ ]:
# Cargar el modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Función para listar sustantivos y nombres propios
def listar_sustantivos_y_nombres_propios(texto):
    doc = nlp(texto)
    # Extraer sustantivos y nombres propios
    sustantivos_y_nombres_propios = [token.text for token in doc if token.pos_ in ['NOUN', 'PROPN']]
    return sustantivos_y_nombres_propios

# Función para procesar el diccionario 'muestra' y extraer sustantivos y nombres propios
def procesar_muestra(muestra):
    for titulo, contenido in muestra.items():
        # Listar sustantivos y nombres propios en el contenido del cuento
        sustantivos_y_nombres_propios = listar_sustantivos_y_nombres_propios(contenido)
        # Mostrar el nombre del cuento y sus sustantivos y nombres propios
        print(f"\nCuento: {titulo}")
        print("Sustantivos y nombres propios:", sustantivos_y_nombres_propios)

procesar_muestra(muestra)



Cuento: LOS DUENDES DEL CUZCO
Sustantivos y nombres propios: ['virrey', 'poeta', 'entendía', 'justicia', 'tradición', 'autoridad', 'relato', 'pueblo', 'mención', 'manuscrito', 'apuntación', 'época', 'salinas', 'línea', 'tiempo', 'gobierno', 'príncipe', 'diablo', 'almirante', 'castilla', 'luz', 'líneas', 'señor', 'suceso', 'don', 'francisco', 'borja', 'aragón', 'circunstancia', 'carácter', 'palabra', 'leyenda', 'salvedad', 'descargo', 'conciencia', 'punto', 'materia', 'don', 'francisco', 'borja', 'aragón', 'príncipe', 'esquilache', 'conde', 'mayalde', 'madrid', 'orden', 'santiago', 'montés', 'año', 'felipe', 'iii', 'virrey', 'perú', 'nombramiento', 'verso', 'desafío', 'regio', 'costumbre', 'murmuración', 'verdad', 'cabeza', 'brazo', 'monarca', 'filibustera', 'juan', 'mendoza', 'luna', 'marqués', 'brío', 'juventud', 'jorge', 'spitberg', 'escuadra', 'costa', 'chile', 'callao', 'encuentro', 'julio', 'seisciento', 'hora', 'combate', 'cerro', 'capitana', 'nave', 'pirata', 'vencedor', 'prisi

#9 Implementar los siguientes módulos:

## Obtener las 5 palabras con mayor peso en cada documento.

In [ ]:
corpus = list(muestra.values())

In [ ]:
def obtener_top5_palabras_por_documento(corpus):
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)

    # Obtener nombres de características (palabras)
    vocabulario = vectorizer.get_feature_names_out()

    # Crear un DataFrame para almacenar los pesos TF-IDF
    df_tfidf = pd.DataFrame(X.toarray(), columns=vocabulario)

    # Obtener las 5 palabras con mayor peso en cada documento
    top5_palabras_documentos = {}
    for i, doc in enumerate(df_tfidf.index):
        # Obtener los pesos del documento actual
        doc_weights = df_tfidf.loc[doc]
        # Obtener las 5 palabras con mayor peso
        top5_palabras = doc_weights.nlargest(5).index.tolist()
        top5_palabras_documentos[doc] = top5_palabras

    # Crear un DataFrame para los resultados de los documentos
    df_top5_documentos = pd.DataFrame(list(top5_palabras_documentos.items()), columns=['Documento', 'Top 5 Palabras'])
    return df_top5_documentos


In [ ]:
# Obtener las 5 palabras con mayor peso en cada documento
df_top5_documentos = obtener_top5_palabras_por_documento(corpus)

# Mostrar resultados
print("\nTop 5 Palabras en Cada Documento:")
print(df_top5_documentos)


Top 5 Palabras en Cada Documento:
   Documento                                     Top 5 Palabras
0          0         [príncipe, esquilache, gusto, laza, cuzco]
1          1      [chinchón, doctor, virreín, atacado, cabrera]
2          2        [laycacoto, mina, carmar, castro, cubierto]
3          3   [castellar, quitar, villega, alguacil, presidio]
4          4   [monclova, cristal, ejemplo, espalda, evangelín]
5          5  [abandonar, costar, custodia, esfuerzo, esmera...
6          6        [amat, alférez, michel, pulido, regimiento]
7          7        [hospital, lego, letra, tierra, cementerio]
8          8     [arriagar, tupacamaru, aviso, cobro, palabras]
9          9  [aquilino, benedicta, benedicto, conducta, for...


##Determinar las 10 palabras con mayor peso en todo el corpus.

In [ ]:
def obtener_top10_palabras_corpus(corpus):
    # Crear el vectorizador TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)

    # Obtener nombres de características (palabras)
    vocabulario = vectorizer.get_feature_names_out()

    # Crear un DataFrame para almacenar los pesos TF-IDF
    df_tfidf = pd.DataFrame(X.toarray(), columns=vocabulario)

    # Determinar las 10 palabras con mayor peso en todo el corpus
    # Sumar los pesos de todas las palabras en todos los documentos
    peso_total_palabras = df_tfidf.sum()
    # Obtener las 10 palabras con mayor peso total
    top10_palabras_corpus = peso_total_palabras.nlargest(10)

    # Crear un DataFrame para los resultados del corpus
    df_top10_corpus = pd.DataFrame(top10_palabras_corpus, columns=['Peso Total'])
    return df_top10_corpus


In [ ]:
# Obtener las 10 palabras con mayor peso en todo el corpus
df_top10_corpus = obtener_top10_palabras_corpus(corpus)
print("\nTop 10 Palabras en Todo el Corpus:")
print(df_top10_corpus)


Top 10 Palabras en Todo el Corpus:
         Peso Total
él         0.484583
virrey     0.476787
hacer      0.472577
decir      0.431737
después    0.426006
don        0.422574
mil        0.405719
dar        0.366823
año        0.333325
fué        0.332471


# Implementar un módulo que liste los personajes de un determinado relato.

In [ ]:
# Función para listar los personajes de un relato (entidades de tipo 'PERSON')
def listar_personajes(texto):
    # Procesar el texto con el modelo de spaCy
    doc = nlp(texto)

    # Extraer los personajes (entidades de tipo 'PERSON')
    personajes = [ent.text for ent in doc.ents if ent.label_ == 'PER']

    return personajes

# Función para mostrar el relato y sus personajes
def mostrar_relato_y_personajes(titulo, contenido):
    personajes = listar_personajes(contenido)
    print(f"\nRelato: {titulo}")
    print("Personajes:", personajes)

# Ejemplo de uso:
# Asumiendo que 'muestra' es tu diccionario de cuentos
def procesar_muestra_y_mostrar_personajes(muestra):
    for titulo, contenido in muestra.items():
        mostrar_relato_y_personajes(titulo, contenido)


In [ ]:
procesar_muestra_y_mostrar_personajes(muestra)



Relato: LOS DUENDES DEL CUZCO
Personajes: ['francisco borja', 'francisco borja aragón', 'santiago montés', 'juan mendoza luna', 'jorge spitberg escuadra holandés', 'poemo santo rosa', 'jacobo', 'alejandro rodrigo borgia san duque', 'lope quevedo', 'tirso molín alarcón moreto', 'diego fernández córdova', 'iglesio allí', 'pedro yarpe montenegro', 'luis puente', 'insigne fray baldomero', 'latino moreyra', 'manuel castilla', 'gabriel vázquez', 'juan gonzález', 'héroe tradición', 'cristo', 'carlos terminado oración', 'poético sucedidoreponer', 'demás cuervo temis']

Relato: LOS POLVOS DE LA CONDESA
Personajes: ['crónico época', 'ignacio lapuente', 'término', 'hallábansar excelentísimo señor don luis jerónimo fernández', 'felipe', 'marqués', 'guant', 'francisco', 'tercián inca endémico valle rimac', 'ignacio loyola', 'luis jerónimo fernández cabrera', 'criptán caballeros santiago', 'conde chinchón', 'holandés', 'callao escuadro', 'felipe cortesanos américa contribuir', 'bombón cayllomar', '

#Implementar un módulo que solicite un texto y muestre un relato relacionado con dicho texto

In [ ]:
# Función para normalizar los títulos (convertir a minúsculas)
def normalizar_titulo(titulo):
    return titulo.lower()

# Función para encontrar un relato relacionado con un texto dado
def encontrar_relato_relacionado(texto, muestra, umbral=0.3):
    # Normalizar solo el texto de entrada
    texto_normalizado = texto.lower()

    # Crear un nuevo corpus que combine títulos y contenidos de los relatos
    titulos = list(muestra.keys())
    corpus = [normalizar_titulo(titulo) + " " + contenido for titulo, contenido in muestra.items()]

    # Vectorizar usando TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus + [texto_normalizado])

    # Calcular la similitud coseno entre el texto proporcionado y los relatos
    similitudes = cosine_similarity(X[-1:], X[:-1])

    # Encontrar el índice del relato con mayor similitud
    indice_relato = np.argmax(similitudes)
    mayor_similitud = similitudes[0, indice_relato]

    # Verificar si la similitud es mayor que el umbral
    if mayor_similitud >= umbral:
        relato_encontrado = titulos[indice_relato]
        contenido_relato = muestra[relato_encontrado]
        return relato_encontrado, contenido_relato
    else:
        return None, None


# Función para solicitar un texto y mostrar el relato relacionado
def solicitar_texto_y_mostrar_relato(muestra, umbral=0.0115):
    # Solicitar el texto
    texto = input("Introduce un texto relacionado con el relato: ")

    # Encontrar el relato relacionado
    titulo, contenido = encontrar_relato_relacionado(texto, muestra, umbral)

    # Mostrar el relato si se encuentra
    if titulo and contenido:
        print(f"\nRelato encontrado:\nTítulo: {titulo}\nCuerpo: {contenido}")
    else:
        print("\nNo se encontró ningún relato relacionado con el texto proporcionado.")


El umbral usado fue de 0.0115, sin embargo, este es para la muestra de los 10 primeros relaos, por lo que esta puede variar en caso se use mas o menos.

In [ ]:
solicitar_texto_y_mostrar_relato(muestra)


Introduce un texto relacionado con el relato: El abuelo y el padre de Evangelina

Relato encontrado:
Título: AMOR DE MADRE
Cuerpo: crónico época virrey brazo plata juán manuela gorriti juzgar conveniente alterar nombre principal personaje tradición pecado venial cometido emplazado alguno significar cuidar falsear verdad histórico bien barruntacer lector razón poderoso desbautizar prójimos agosto mil seisciento noventa hacer entrada limo excelentísimo señor don melchor portocarrero lazo vega conde monclova comendador zarza orden alcántara vigésimo tercio virrey perú majestad carlos además hijo doña josefa familia servidumbre acompañábanlo méxico cuyo gobierno fué trasladar reino soldado español distinguíar bizarro marcial aspecto fernando vergar hijodalgo extremeño capitán gentileshombres lanzas contábar belleza mexicano dejar reputación austerir monje benedictino pendenciero jugador amante dar guerra mujer difícil él sentar cabeza profesar paternal afecto proponer casar mano ver result

In [ ]:
solicitar_texto_y_mostrar_relato(muestra)


Introduce un texto relacionado con el relato: El profesor Lauro conoce a Messi

No se encontró ningún relato relacionado con el texto proporcionado.
